# 一般のロバスト方策勾配の収束について

**注意：めっちゃ頑張ったけど(31)式付近が間違ってる気がする．多分min Pが複数個ある場合に証明ができない．なので今回は$\min_P$の数が１個であることを仮定します．**

参考：
* [Policy Gradient in Robust MDPs with Global Convergence Guarantee](https://arxiv.org/abs/2212.10439)

[RL_robust_sa_gradient.ipynb](RL_robust_sa_gradient.ipynb)でやったロバストMDPに対する方策勾配法はSA-rectangularでしか成立しません．
今回は一般のCompactなUncertainty setに対して成り立つ収束について見ていきます．
表記は論文に任せます．

ロバストMDPでは次の最適化問題を解くことを考えます：

$$
\min _{\boldsymbol{\pi} \in \Pi} \max _{\boldsymbol{p} \in \mathcal{P}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p}):=\boldsymbol{\rho}^{\top} \boldsymbol{v}^{\boldsymbol{\pi}, \boldsymbol{p}}=\sum_{s \in \mathcal{S}} \rho_s v_s^{\boldsymbol{\pi}, \boldsymbol{p}}
$$

また，Occupancy measureを
$$d_{\boldsymbol{\rho}}^{\boldsymbol{\pi}, \boldsymbol{p}}\left(s^{\prime}\right)=(1-\gamma) \sum_{s \in \mathcal{S}} \sum_{t=0}^{\infty} \gamma^t \rho(s) p_{s s^{\prime}}^{\boldsymbol{\pi}}(t)$$
で定義します．
また，簡単のために次の表記も導入しておきます．

$$
\min _{\boldsymbol{\pi} \in \Pi}\left\{\Phi(\boldsymbol{\pi}):=\max _{\boldsymbol{p} \in \mathcal{P}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})\right\} .
$$

あとはこの$\Phi(\pi)$について勾配降下法をすればいい気がしますが，ことはそう単純ではありません．
まず，$\Phi$は一般には微分可能ではなく，さらに凸でも凹でもないので，一般には劣微分も存在しない場合があります（論文では[Non-convex Min-Max Optimization: Applications, Challenges, and Recent Theoretical Advances](https://arxiv.org/abs/2006.08141)あたりを根拠にしていますが，[RL_robust_sa_gradient.ipynb](RL_robust_sa_gradient.ipynb)でみたようにフレシェ劣微分の意味で至るところ微分可能な気がする）．


---

**補題：Danskinの定理**

準備：
* $\mathcal{Z} \subseteq \mathbb{R}^m$をコンパクトな集合とします．
* $\phi: \mathbb{R}^n \times \mathcal{Z} \to \mathbb{R}$を連続な関数とします．
* $\phi(\cdot, z): \mathbb{R}^n \to \mathbb{R}$はそれぞれの$z \in \mathcal{Z}$ で凸とします．
  
もし$\phi(\cdot, z)$が全ての$z \in \mathcal{Z}$で微分可能かつ$\nabla\phi(x, \cdot)$がそれぞれの$x$について$\mathcal{Z}$ で連続ならば，$f(\boldsymbol{x}):=\max _{z \in \mathcal{Z}} \phi(\boldsymbol{x}, z)$とすると，任意の$x \in \mathbb{R}^n$で

$$
\partial f(\boldsymbol{x})=\operatorname{conv}\left\{\nabla_x \phi(\boldsymbol{x}, z) \mid z \in \underset{z \in \mathcal{Z}}{\arg \max } \phi(\boldsymbol{x}, z)\right\}
$$

つまり，各$x$における勾配は，各$\argmax_z$での勾配の重み付き和で書けます．

## ロバスト方策勾配

最悪ケースの遷移$p_t$は
$$
J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right) = \max _{\boldsymbol{p} \in \mathcal{P}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}\right)
$$
を満たすとします（論文中では誤差がある場合を考えてますが，今回は面倒なので考えません）．

これについて，次の方策更新を考えます．

$$
\begin{aligned}
\boldsymbol{\pi}_{t+1} & \in \arg \min _{\boldsymbol{\pi} \in \Pi}\left\langle\nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right), \boldsymbol{\pi}-\boldsymbol{\pi}_t\right\rangle+\frac{1}{2 \alpha_t}\left\|\boldsymbol{\pi}-\boldsymbol{\pi}_t\right\|^2 \\
& =\operatorname{Proj}_{\Pi}\left(\boldsymbol{\pi}_t-\alpha_t \nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right)\right)
\end{aligned}
$$

これは各状態について考えると

$$
\boldsymbol{\pi}_{t+1, s}=\operatorname{Proj}_{\Delta A}\left(\boldsymbol{\pi}_{t, s}-\alpha_t \nabla_{\boldsymbol{\pi}_s} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right)\right), \quad \forall s \in \mathcal{S}
$$

として書けます．
$p_t$が固定されたときの勾配は通常のMDPの勾配と同じなので，

$$
\frac{\partial J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})}{\partial \pi_{s a}}=\frac{1}{1-\gamma} \cdot d_{\boldsymbol{\rho}}^{\boldsymbol{\pi}, \boldsymbol{p}}(s) \cdot q_{s a}^{\boldsymbol{\pi}, \boldsymbol{p}} .
$$

で与えられます．

### 1. $J$と$\Phi$のリプシッツ性，Smoothness，Convexityの証明

$J_\rho(\boldsymbol{\pi}, \boldsymbol{p})$は$L_\pi$-リプシッツであり，$\ell_\pi$-smoothです．ここで

$$
L_{\boldsymbol{\pi}}:=\frac{\sqrt{A}}{(1-\gamma)^2}, \quad \ell_{\boldsymbol{\pi}}:=\frac{2 \gamma A}{(1-\gamma)^3}
$$

としました．また，$\Phi(\pi)$は$L_\pi$-リプシッツであり，$\ell_\pi$-weakly-convexです．

**証明**　

$\ell_\pi$-smoothの証明は簡単なので飛ばします．
$L_\pi$-リプシッツの証明をしましょう．

$$
\begin{aligned}
\left\|\nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})\right\| & =\sqrt{\sum_{s, a}\left(\frac{\partial J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})}{\partial \pi_{s a}}\right)^2} \\
& =\frac{1}{1-\gamma} \sqrt{\sum_a \sum_s\left(d_{\boldsymbol{\rho}}^{\boldsymbol{\pi}, \boldsymbol{p}}(s) q_{s a}^{\boldsymbol{\pi}, \boldsymbol{p}}\right)^2} \\
& \leq \frac{1}{(1-\gamma)^2} \sqrt{\sum_a \sum_s\left(d_{\boldsymbol{\rho}}^{\boldsymbol{\pi}, \boldsymbol{\xi}}(s)\right)^2} \leq \frac{\sqrt{A}}{(1-\gamma)^2},
\end{aligned}
$$
が成立します．これが$L_\pi$リプシッツの証明そのものです．

続いて$\Phi$の証明をしましょう．
$$
\begin{aligned}
\Phi\left(\boldsymbol{\pi}_1\right)-\Phi\left(\boldsymbol{\pi}_2\right) & =\max _{\boldsymbol{p} \in \mathcal{P}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_1, \boldsymbol{p}\right)-\max _{\boldsymbol{p} \in \mathcal{P}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_2, \boldsymbol{p}\right) \\
& =J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_1, \boldsymbol{p}_1\right)-J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_2, \boldsymbol{p}_2\right) \\
& \leq J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_1, \boldsymbol{p}_1\right)-J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_2, \boldsymbol{p}_1\right) \\
& \leq L_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_1-\boldsymbol{\pi}_2\right\| .
\end{aligned}
$$
なので，$\Phi$は$L_\pi$リプシッツです．

$\ell_\pi$-weakly convexの証明は[Efficient Algorithms for Smooth Minimax Optimization](https://arxiv.org/abs/1907.01543)のLemma 3に載っています．

<!-- ---

### 2. $J$についてのGradient dominanceの証明

[RL_policy_gradient_convergence_rate.ipynb](RL_policy_gradient_convergence_rate.ipynb)でやったように，次のGradient dominanceが成立します：

$$
J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})-J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}^{\star}, \boldsymbol{p}\right) \leq \frac{D}{1-\gamma} \max _{\overline{\boldsymbol{\pi}} \in \Pi}(\boldsymbol{\pi}-\overline{\boldsymbol{\pi}})^{\top} \nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})
$$ -->


### 2. $J$についてのGradient dominanceの証明

[RL_policy_gradient_convergence_rate.ipynb](RL_policy_gradient_convergence_rate.ipynb)でやったように，次のGradient dominanceが成立します：

$$
J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})-J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}^{\star}, \boldsymbol{p}\right) \leq \frac{D}{1-\gamma} \max _{\overline{\boldsymbol{\pi}} \in \Pi}(\boldsymbol{\pi}-\overline{\boldsymbol{\pi}})^{\top} \nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})
$$

### 3. $\Phi$のGradient dominance

$\Phi_\lambda(\pi)$は$\Phi(\pi)$のモーロー包とし，$D:=\sup _{\boldsymbol{\pi} \in \Pi, \boldsymbol{p} \in \mathcal{P}}\left\|\boldsymbol{d}_{\boldsymbol{\rho}}^{\boldsymbol{\pi}, \boldsymbol{p}} / \boldsymbol{\rho}\right\|_{\infty}<\infty$とします．

さて，上でやったように，$J_\rho(\pi, p)$は$\pi$について$\ell_\pi$-weakly convexです．
よって，

$$
\tilde{J}_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p}):=J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})+\frac{\ell_{\boldsymbol{\pi}}}{2}\|\boldsymbol{\pi}\|^2
$$

が$\pi$について凸であり，$\nabla_{\boldsymbol{\pi}} \tilde{J}_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})=\nabla_{\boldsymbol{\pi}} J_\rho(\boldsymbol{\pi}, \boldsymbol{p})+\ell_{\boldsymbol{\pi}} \boldsymbol{\pi}$です．
$\tilde{\Phi}(\pi):=\max _{\boldsymbol{p} \in \mathcal{P}} \tilde{J}_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})$
とします．

上で述べたように，最悪ケースの$P$は唯一であるとします．
$\tilde{J}$の凸性と$\partial \tilde{\Phi}(\pi)=\partial \Phi(\pi) + \ell_\pi \pi$ を使えば，Danskinの定理から

$$
\begin{aligned}
\nabla \tilde{\Phi}(\boldsymbol{\pi}) & =\nabla_{\boldsymbol{\pi}} \tilde{J}_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p}) \text{ where } \boldsymbol{p} = \underset{\boldsymbol{p} \in \mathcal{P}}{\arg \max } \tilde{J}_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p}) \\
\Longrightarrow \nabla \Phi(\boldsymbol{\pi})+\ell_{\boldsymbol{\pi}} \boldsymbol{\pi} & =\nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})+\ell_{\boldsymbol{\pi}} \boldsymbol{\pi} \text{ where } \boldsymbol{p} = \underset{\boldsymbol{p} \in \mathcal{P}}{\arg \max } J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})\\
\Longrightarrow \nabla \Phi(\boldsymbol{\pi}) & =\nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p}) \text{ where } \boldsymbol{p} = \underset{\boldsymbol{p} \in \mathcal{P}}{\arg \max } J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p}) .
\end{aligned}
$$
が成り立ちます．３行目では$\ell_\pi \pi$が$p$に依存しないので消しています．


$\tilde{\boldsymbol{\pi}}=\arg \min _{\hat{\boldsymbol{\pi}} \in \Pi} \Phi(\hat{\boldsymbol{\pi}})+\ell_{\boldsymbol{\pi}}\|\boldsymbol{\pi}-\hat{\boldsymbol{\pi}}\|^2$としましょう（これはMoreau Yoshida Regularizationみたいなやつです多分）．
また，
$$\tilde{p} = \arg \max _{\boldsymbol{p} \in \mathcal{P}} J_\rho(\boldsymbol{\tilde{\pi}}, \boldsymbol{p})$$
を$\tilde{\pi}$における最悪ケースの遷移とします．

---

**補足**：

**論文中では集合を考えていますが，それだと多分無理です．なので，唯一存在することを仮定します．これは例えば[Robust Markov Decision Processes without Model Estimation](https://arxiv.org/abs/2302.01248)などの正則化を使えば成立するはずです．TODO: ちゃんと確かめてませんが，例えば$p$についてのエントロピー正則化項を追加して，その係数のゼロ極限$\to 0$を考えるなどで大丈夫かも？**

唯一存在する場合は（フレシェ）微分可能なので，Moreau envelopeを使った面倒なことしなくていい気もしますが，ちょっとめんどくさいかもしれません．
微分可能でもsmoothであるとは限りません．例えば
$$
g(t)= \begin{cases}t|t|^{1 / 2} \cos \frac{1}{t} & \text { if } t \neq 0 \\ 0 & \text { if } t=0\end{cases}
$$
は微分可能ですが，その微分
$$
g^{\prime}(t)=\left\{\begin{array}{l}
|t|^{1 / 2} \cos \frac{1}{t}-\frac{|t|^{1 / 2}}{t} \sin \frac{1}{t} \\
0
\end{array}\right.
$$
は$t=0$付近で無限に飛び，Lipschitz連続ではありません．

ただし２階微分が有限であれば１階微分はLipschitz連続になるので，方策勾配の２階微分などを考えるといけるかも？しれません．
[Hessian Aided Policy Gradient](https://proceedings.mlr.press/v97/shen19d.html)あたりの議論を使えばできるかもですが，今回はめんどくさいのでMoreau envelopeのやり方で回避します．

---

このとき，ロバストMDPの方策勾配は
$$
\tilde{\boldsymbol{\xi}}= \nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}\left(\tilde{\boldsymbol{\pi}}, \tilde{\boldsymbol{p}}\right)
$$
で表されます．

JについてのGradient dominanceを使えば
$$
\begin{aligned}
\Phi(\tilde{\boldsymbol{\pi}})-\Phi\left(\boldsymbol{\pi}^{\star}\right) 
& \leq J(\tilde{\boldsymbol{\pi}}, \tilde{p})-J(\pi^\star, \tilde{p}) \\
&\leq 
\frac{D}{1-\gamma}
\max _{\boldsymbol{\pi}' \in \Pi}\left\langle\tilde{\boldsymbol{\pi}} - \boldsymbol{\pi}',\nabla_{\boldsymbol{\pi}} J\left(\tilde{\boldsymbol{\pi}}, \tilde{\boldsymbol{p}}\right)\right\rangle\\
&=
\frac{D}{1-\gamma}
\max _{\boldsymbol{\pi}' \in \Pi}\left\langle\tilde{\boldsymbol{\pi}} - \boldsymbol{\pi}',\tilde{\boldsymbol{\xi}}\right\rangle
\end{aligned}
$$
が成立します．
ここでさらに[CVX_weakly_convex_and_Moreau_envelope.ipynb](CVX_weakly_convex_and_Moreau_envelope.ipynb)
の補題を使うと，任意の$\pi' \in \Pi'$について，
$$
\begin{aligned}
\frac{D}{1-\gamma}
\left\langle\tilde{\boldsymbol{\pi}} - \boldsymbol{\pi}',\tilde{\boldsymbol{\xi}}\right\rangle
&\leq 
\frac{D}{1-\gamma}
\langle\nabla \Phi_{\frac{1}{2\ell_\pi}(\pi)}, \hat{\pi} - \pi' \rangle \\
&\leq 
\frac{D}{1-\gamma}
\left\|\nabla \Phi_{\frac{1}{2\ell_\pi}(\pi)}\right\|_2 \left\|\hat{\pi} - \pi'\right\|_2
\end{aligned}
$$
が成立します．２つ目の不等式はコーシー・シュワルツです．
ここで，[分布の差についてのL1ノルムの不等式](https://math.stackexchange.com/questions/3835146/l1-norm-of-difference-between-two-probability-distributions-over-a-finite-set)を使うと，

$$
\begin{aligned}
\left\|\hat{\pi} - \pi'\right\|_2&=
\sqrt{
\sum_{s} \sum_{a} \left(\hat{\pi}(a \mid s) - \pi'(a \mid s)\right)^2
}\\
&\leq 
\sqrt{S} 
\max_{s}\sqrt{
\sum_{a} \left(\hat{\pi}(a \mid s) - \pi'(a \mid s)\right)^2
}\\
&\leq 
\sqrt{S} 
\max_{s}
\sum_{a} \left|\hat{\pi}(a \mid s) - \pi'(a \mid s)\right|
\leq 2\sqrt{S}
\end{aligned}
$$
なので，結局
$$
\begin{aligned}
\Phi(\tilde{\pi}) - \Phi(\pi^\star)
&\leq 
\frac{2\sqrt{S}D}{1-\gamma}
\left\|\nabla \Phi_{\frac{1}{2\ell_\pi}(\pi)}\right\|_2 
\end{aligned}
$$
です．

あとはLipschitz性と合わせれば

$$
\begin{aligned}
\Phi(\boldsymbol{\pi})-\Phi\left(\boldsymbol{\pi}^{\star}\right) & =\Phi(\boldsymbol{\pi})-\Phi(\tilde{\boldsymbol{\pi}})+\Phi(\tilde{\boldsymbol{\pi}})-\Phi\left(\boldsymbol{\pi}^{\star}\right) \\
& \leq \frac{2 D \sqrt{S}}{1-\gamma}\left\|\nabla \Phi_{\frac{1}{2 \ell_\pi}}(\boldsymbol{\pi})\right\|+\Phi(\boldsymbol{\pi})-\Phi(\tilde{\boldsymbol{\pi}}) \\
& \leq \frac{2 D \sqrt{S}}{1-\gamma}\left\|\nabla \Phi_{\frac{1}{2 \ell}}(\boldsymbol{\pi})\right\|+L_{\boldsymbol{\pi}}\|\boldsymbol{\pi}-\tilde{\boldsymbol{\pi}}\| \\
& =\frac{2 D \sqrt{S}}{1-\gamma}\left\|\nabla \Phi_{\frac{1}{2 \ell \pi}}(\boldsymbol{\pi})\right\|+L_{\boldsymbol{\pi}} \cdot \frac{\left\|\nabla \Phi_{\frac{1}{2 \ell \pi}}(\boldsymbol{\pi})\right\|}{2 \ell_{\boldsymbol{\pi}}}\\
& =\left(\frac{2 D \sqrt{S}}{1-\gamma} + \frac{L_\pi}{2\ell_\pi}\right)\left\|\nabla \Phi_{\frac{1}{2 \ell \pi}}(\boldsymbol{\pi})\right\|
\end{aligned}
$$
が成り立ちます．

### 4. $\varepsilon$-stationary pointに収束することの証明

$\tilde{\boldsymbol{\pi}}_t=\arg \min _{\tilde{\boldsymbol{\pi}} \in \Pi} \Phi(\tilde{\boldsymbol{\pi}})+\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_t-\tilde{\boldsymbol{\pi}}\right\|^2$としましょう．
また，$\pi_t$についての最悪ケースを$p_t$とし，
$$
\begin{aligned}
\boldsymbol{\pi}_{t+1} & \in \arg \min _{\boldsymbol{\pi} \in \Pi}\left\langle\nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right), \boldsymbol{\pi}-\boldsymbol{\pi}_t\right\rangle+\frac{1}{2 \alpha_t}\left\|\boldsymbol{\pi}-\boldsymbol{\pi}_t\right\|^2 \\
& =\operatorname{Proj}_{\Pi}\left(\boldsymbol{\pi}_t-\alpha_t \nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right)\right)
\end{aligned}
$$
であることを思い出しましょう．


このとき，

$$
\begin{aligned}
\Phi_{\frac{1}{2 \ell_\pi}}\left(\boldsymbol{\pi}_{t+1}\right) & =\min _{\boldsymbol{\pi}} \Phi(\boldsymbol{\pi})+\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_{t+1}-\boldsymbol{\pi}\right\|^2 \\
& \leq \Phi\left(\tilde{\boldsymbol{\pi}}_t\right)+\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_{t+1}-\tilde{\boldsymbol{\pi}}_t\right\|^2 \\
& =\Phi\left(\tilde{\boldsymbol{\pi}}_t\right)+\ell_{\boldsymbol{\pi}}\left\|\mathcal{P}_{\Pi}\left(\boldsymbol{\pi}_t-\alpha \nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right)\right)-\mathcal{P}_{\Pi}\left(\tilde{\boldsymbol{\pi}}_t\right)\right\|^2 \\
& \leq \Phi\left(\tilde{\boldsymbol{\pi}}_t\right)+\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_t-\alpha \nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right)-\tilde{\boldsymbol{\pi}}_t\right\|^2 \text{ Projしたほうがノルムは小さくなる }\\
& =\underbrace{\Phi\left(\tilde{\boldsymbol{\pi}}_t\right)+\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_t-\tilde{\boldsymbol{\pi}}_t\right\|^2}_{\Phi_{\frac{1}{2\ell_\pi}}(\pi_t)}-2 \ell_{\boldsymbol{\pi}} \alpha\left(\nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right), \boldsymbol{\pi}_t-\tilde{\boldsymbol{\pi}}_t\right\rangle+\alpha^2 \ell_{\boldsymbol{\pi}} \underbrace{\left\|\nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right)\right\|^2}_{\leq L_\pi^2 \;\text{ due to Lipschitz}} \\
& \leq \Phi_{\frac{1}{2 \ell_\pi}}\left(\boldsymbol{\pi}_t\right)+2 \ell_{\boldsymbol{\pi}} \alpha\left(\Phi\left(\tilde{\boldsymbol{\pi}}_t\right)-\Phi\left(\boldsymbol{\pi}_t\right)+\frac{\ell_{\boldsymbol{\pi}}}{2}\left\|\boldsymbol{\pi}_t-\tilde{\boldsymbol{\pi}}_t\right\|^2\right)+\alpha^2 \ell_{\boldsymbol{\pi}} L_{\boldsymbol{\pi}}^2,
\end{aligned}
$$

ここで最後の不等式は次の不等式を利用してます（$\Phi$が$\ell_\pi$-weakly convexityであることを使ってます．）

$$
\begin{aligned}
\Phi\left(\tilde{\boldsymbol{\pi}}_t\right) \geq J_{\boldsymbol{\rho}}\left(\tilde{\boldsymbol{\pi}}_t, \boldsymbol{p}_t\right) & \geq J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right)+\left\langle\nabla_{\boldsymbol{\pi}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right), \tilde{\boldsymbol{\pi}}_t-\boldsymbol{\pi}_t\right\rangle-\frac{\ell_{\boldsymbol{\pi}}}{2}\left\|\boldsymbol{\pi}_t-\tilde{\boldsymbol{\pi}}_t\right\|^2 
\end{aligned}
$$

です．
上でやった不等式を$t$について和を取ると

$$
\Phi_{\frac{1}{2 \ell \pi}}\left(\boldsymbol{\pi}_{T-1}\right) \leq \Phi_{\frac{1}{2 \ell \pi}}\left(\boldsymbol{\pi}_0\right)+2 \ell_{\boldsymbol{\pi}} \alpha \sum_{t=0}^{T-1}\left(\Phi\left(\tilde{\boldsymbol{\pi}}_t\right)-\Phi\left(\boldsymbol{\pi}_t\right)+\frac{\ell \boldsymbol{\pi}}{2}\left\|\boldsymbol{\pi}_t-\tilde{\boldsymbol{\pi}}_t\right\|^2\right)+T \alpha^2 \ell_{\boldsymbol{\pi}} L_{\boldsymbol{\pi}}^2
$$

です．変形して

$$
\sum_{t=0}^{T-1}\left(\Phi\left(\boldsymbol{\pi}_t\right)-\Phi\left(\tilde{\boldsymbol{\pi}}_t\right)-\frac{\ell_{\boldsymbol{\pi}}}{2}\left\|\boldsymbol{\pi}_t-\tilde{\boldsymbol{\pi}}_t\right\|^2\right) \leq \frac{\Phi_{\frac{1}{2 \ell \pi}}\left(\boldsymbol{\pi}_0\right)-\Phi_{\frac{1}{2 \ell}}\left(\boldsymbol{\pi}_{T-1}\right)}{2 \ell_{\boldsymbol{\pi}} \alpha}+\frac{T \alpha L_\pi^2}{2}.
$$

が得られます．また，凸関数の性質を使えば

$$
\begin{aligned}
&\begin{aligned}
& \Phi\left(\boldsymbol{\pi}_t\right)-\Phi\left(\tilde{\boldsymbol{\pi}}_t\right)-\frac{\ell_{\boldsymbol{\pi}}}{2}\left\|\boldsymbol{\pi}_t-\tilde{\boldsymbol{\pi}}_t\right\|^2 \\
& =\Phi\left(\boldsymbol{\pi}_t\right)+\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_t-\boldsymbol{\pi}_t\right\|^2-\Phi\left(\tilde{\boldsymbol{\pi}}_t\right)-\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_t-\tilde{\boldsymbol{\pi}}_t\right\|^2+\frac{\ell_{\boldsymbol{\pi}}}{2}\left\|\boldsymbol{\pi}_t-\tilde{\boldsymbol{\pi}}_t\right\|^2 \\
& =\Phi\left(\boldsymbol{\pi}_t\right)+\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_t-\boldsymbol{\pi}_t\right\|^2-\min _{\boldsymbol{\pi} \in \Pi}\left\{\Phi(\boldsymbol{\pi})+\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_t-\boldsymbol{\pi}\right\|^2\right\}+\frac{\ell_{\boldsymbol{\pi}}}{2}\left\|\boldsymbol{\pi}_t-\tilde{\boldsymbol{\pi}}_t\right\|^2
\end{aligned}\\
&\stackrel{(a)}{\geq} \ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_t-\tilde{\boldsymbol{\pi}}_t\right\|^2=\frac{1}{4 \ell_{\boldsymbol{\pi}}}\left\|\nabla \Phi_{\frac{1}{2 \ell \pi}}\left(\boldsymbol{\pi}_t\right)\right\|^2 .
\end{aligned}
$$
が成立します．最後の式はモーロー包の勾配を使えば出てきます．

$\overline{\boldsymbol{\pi}}_1:=\arg \min _{\overline{\boldsymbol{\pi}} \in \Pi} \Phi(\overline{\boldsymbol{\pi}})+\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_1-\overline{\boldsymbol{\pi}}\right\|^2$ and $\overline{\boldsymbol{\pi}}_2:=\arg \min _{\overline{\boldsymbol{\pi}} \in \Pi} \Phi(\overline{\boldsymbol{\pi}})+\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_2-\overline{\boldsymbol{\pi}}\right\|^2$
とします．
このとき，

$$
\begin{aligned}
\Phi_{\frac{1}{2 \ell \pi}}\left(\boldsymbol{\pi}_1\right)-\Phi_{\frac{1}{2 \ell \pi}}\left(\boldsymbol{\pi}_2\right) & =\min _{\overline{\boldsymbol{\pi}} \in \Pi}\left(\Phi(\overline{\boldsymbol{\pi}})+\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_1-\overline{\boldsymbol{\pi}}\right\|^2\right)-\min _{\overline{\boldsymbol{\pi}} \in \Pi}\left(\Phi(\overline{\boldsymbol{\pi}})+\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_2-\overline{\boldsymbol{\pi}}\right\|^2\right) \\
& =\Phi\left(\overline{\boldsymbol{\pi}}_1\right)+\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_1-\overline{\boldsymbol{\pi}}_1\right\|^2-\Phi\left(\overline{\boldsymbol{\pi}}_2\right)-\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_2-\overline{\boldsymbol{\pi}}_2\right\|^2 \\
& \leq \Phi\left(\overline{\boldsymbol{\pi}}_2\right)+\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_1-\overline{\boldsymbol{\pi}}_2\right\|^2-\Phi\left(\overline{\boldsymbol{\pi}}_2\right)-\ell_{\boldsymbol{\pi}}\left\|\boldsymbol{\pi}_2-\overline{\boldsymbol{\pi}}_2\right\|^2 \\
& =\ell_{\boldsymbol{\pi}}\left(\left\|\boldsymbol{\pi}_1-\overline{\boldsymbol{\pi}}_2\right\|^2-\left\|\boldsymbol{\pi}_2-\overline{\boldsymbol{\pi}}_2\right\|^2\right) \\
& \leq 2 \ell_{\boldsymbol{\pi}} S .
\end{aligned}
$$

です．全部合体させると

$$
\sum_{t=0}^{T-1}\left\|\nabla \Phi_{\frac{1}{2 \ell \pi}}\left(\boldsymbol{\pi}_t\right)\right\|^2 \leq \frac{4 \ell_\pi S}{\alpha}+2 T \alpha \ell_\pi L_\pi^2
$$

が得られます．左辺が$T \varepsilon^2$より小さい場合，つまり
$$
T \cdot \min _t\left\|\nabla \Phi_{\frac{1}{2 \ell \pi}}\left(\boldsymbol{\pi}_t\right)\right\|^2 \leq \sum_{t=0}^{T-1}\left\|\nabla \Phi_{\frac{1}{2 \ell \pi}}\left(\boldsymbol{\pi}_t\right)\right\|^2 \leq T \epsilon^2,
$$
のとき，$\pi_{\hat{t}}$が$\varepsilon$-停留点になるような$\hat{t}$が存在します．


### 5. 停留点が$\varepsilon$-最適である証明

ここでは
$$
J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}^{\star}, \boldsymbol{p}^{\star}\right)-\max _{\boldsymbol{p} \in \mathcal{P}} J_{\boldsymbol{\rho}}\left(\boldsymbol{\pi}_t, \boldsymbol{p}\right)=\Phi\left(\boldsymbol{\pi}^{\star}\right)-\Phi\left(\boldsymbol{\pi}_t\right) \leq \epsilon .
$$
であるような$t$が存在することを示します．

上でやったGradient dominanceから，

$$
J\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right)-\min _{\boldsymbol{\pi} \in \Pi} \max _{\boldsymbol{p} \in \mathcal{P}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p}) \leq \Phi\left(\boldsymbol{\pi}_t\right)-\Phi\left(\boldsymbol{\pi}^{\star}\right) \leq 
\left(\frac{2 D \sqrt{S}}{1-\gamma} + \frac{L_\pi}{2\ell_\pi}\right)\left\|\nabla \Phi_{\frac{1}{2 \ell \pi}}(\boldsymbol{\pi}_t)\right\|
$$

であり，変形すれば

$$
\begin{aligned}
\min _{t \in\{0, \cdots, T-1\}}\left\{J\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right)-\min _{\boldsymbol{\pi} \in \Pi} \max _{\boldsymbol{p} \in \mathcal{P}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})\right\} & \leq \frac{1}{T} \sum_{t=0}^{T-1}\left(J\left(\boldsymbol{\pi}_t, \boldsymbol{p}_t\right)-\min _{\boldsymbol{\pi} \in \Pi} \max _{\boldsymbol{p} \in \mathcal{P}} J_{\boldsymbol{\rho}}(\boldsymbol{\pi}, \boldsymbol{p})\right) \\
& \leq \frac{1}{T} \sum_{t=0}^{T-1}\left(\Phi\left(\boldsymbol{\pi}_t\right)-\Phi\left(\boldsymbol{\pi}^{\star}\right)\right) \\
& \leq \frac{1}{T}\left(\frac{2D \sqrt{S}}{1-\gamma}+\frac{L_{\boldsymbol{\pi}}}{2 \ell_{\boldsymbol{\pi}}}\right) \sum_{t=0}^{T-1}\left\|\nabla \Phi_{\frac{1}{2 \ell_{\boldsymbol{\pi}}}}\left(\boldsymbol{\pi}_t\right)\right\|
\end{aligned}
$$

です．コーシー・シュワルツの不等式から

$$
\frac{1}{\sqrt{T}} \sum_{t=0}^{T-1}\left\|\nabla \Phi_{\frac{1}{2 \ell \pi}}\left(\boldsymbol{\pi}_t\right)\right\| \leq \sqrt{\sum_{t=0}^{T-1}\left\|\nabla \Phi_{\frac{1}{2 \ell} \pi}\left(\boldsymbol{\pi}_t\right)\right\|^2} .
$$

です．さらにステップ４の不等式を使えば，

$$
\begin{aligned}
& \leq \frac{1}{\sqrt{T}}\left(\frac{D \sqrt{S}}{1-\gamma}+\frac{L_\pi}{2 \ell_\pi}\right) \sqrt{\sum_{t=0}^{T-1}\left\|\nabla \Phi_{\frac{1}{2 \ell_\pi}}\left(\boldsymbol{\pi}_t\right)\right\|^2} \\
& \leq \frac{1}{\sqrt{T}}\left(\frac{D \sqrt{S}}{1-\gamma}+\frac{L_\pi}{2 \ell_\pi}\right) \sqrt{\left(\frac{4 \ell_\pi S}{\alpha}+2 T \alpha \ell_\pi L_\pi^2\right)} \;.
\end{aligned}
$$

が成立します．あとはこれが$\epsilon$になるように$\alpha$と$T$を調整すれば終わりです．